# ATR Volatility
Explore Average True Range (ATR)–based volatility features and strategies on ETFs (e.g., SPY/QQQ), using the QuantTrade stack (DuckDB → pandas → Backtrader), with standardized metrics and charts.

**Purpose**
Explore how MACD (EMA_fast − EMA_slow) and its Signal and Histogram can drive entries/exits.

Build reusable strategy components:
- `ATRBreakout` (Donchian breakout + ATR trailing stop)
- `ATRPullback` (uptrend + pullback depth in ATRs + EMA reclaim)
- `KeltnerTrend` (price > Keltner upper with optional trend filter)

## Feature Engineering (what we compute)

True Range & ATR
- `tr` = max of: `high-low`, |`high-prev_close`|, |`low-prev_close`|
- `atr14` (Wilder, ewm(alpha=1/n)), natr14 = atr14/close

Keltner Channels (ATR-based)
- `kc20_basis` (EMA/SMA), `kc20_u` = basis + k*atr20, `kc20_l` = basis - k*atr20 (default k=2)

Breakout scaffolding
- `hh_20`, `ll_20` (rolling highest high / lowest low)

ATR trailing stops
- `atr_stop_long_3` = cummax(close - 3*atr14), atr_stop_short_3 (optional)

Diagnostics
- `chop14` (Choppiness Index)
- `bb_bw20` (Bollinger bandwidth)
- `atr_vol_regime` via NATR thresholds (low/med/high)


## Strategies (entries/exits)
1) ATR Breakout
- Entry: close > `hh_20` AND natr14 ≥ natr_min (e.g., 0.8%)
- Exit: close < `ll_20` or ATR trailing stop (atr_stop_long_3)
- Sizer: ATR risk (risk_cash / (m_atr*ATR))

2) Keltner Trend
- Entry: close > `kc20_u` (+ optional trend filter EMA20 > EMA50)
- Exit: close < `kc20_l` or ATR trailing stop

3) ATR Pullback

- Trend: close > SMA200
- Entry: pullback depth ≥ pullback_atr * atr14 from recent high, then close > EMA20
- Exit: close < EMA20 or ATR trailing stop

## Parameters & Defaults
- ATR: `n=14` (Wilder), `natr = atr/close`
- Keltner: `basis=ema`, `n_basis=20`, `n_atr=20`, `k=2.0`
- HH/LL: `window=20`
- ATR Stop: `m=3.0` (`atr_stop_long_3`)
- Regime thresholds (NATR): low < 0.008, med < 0.02, else high
- Sizing: `risk_cash=200`, `m_atr=3.0` (in `ATRRiskSizer`)

**Notebook Sections**
1. **Setup** – initialize environment and configs  
2. **Helpers** – indicator and chart utilities  
3. **Data Ingestion** – load ETF data (DuckDB → Pandas)  
4. **Data Quality Checks** – ensure clean input data  
5. **Feature Engineering** – compute:

```
ema20     ma200 
tr     atr14    natr14  ...  kc20_basis      kc20_u      kc20_l 
hh_20   ll_20  atr_stop_long_3  atr_stop_short_3     chop14
bb_bw20  atr_vol_regime 
```
6. **Strategy** – define and run the ATR in Backtrader  
7. **Performance Evaluation** – analyze returns, drawdowns, Sharpe, and visualize executions  

**Outcomes**
- Reusable feature pack: TR, ATR(n), NATR(n), Keltner channels, HH/LL, ATR trailing stops, CHOP, BB bandwidth, simple regime flags.
- Three strategies:

1. ATR Breakout (Donchian breakout + ATR trailing stop)
2. Keltner Trend (price > Keltner upper with optional trend filter)
3. ATR Pullback (uptrend + pullback depth in ATRs + EMA reclaim)

- Performance template: returns, Sharpe, drawdowns, trade stats, R-distribution, parameter leaderboard.
- Visuals: price with Keltner envelopes, HH/LL, ATR trailing stop, trade markers, optional NATR/CHOP context.


## 1. Setup

In [106]:
import os,sys
import duckdb
from pathlib import Path
import pandas as pd
import json
import backtrader as bt


In [107]:
PROJECT_ROOT = Path.cwd().parents[0]

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"Project Root: {PROJECT_ROOT}")

Project Root: /Users/luyanda/workspace/QuantTrade


In [108]:
from utils.charts import render_lightweight_chart

In [109]:
from utils.duck import to_bt_daily_duckdb, to_bt_minute_duckdb

In [110]:
from utils.features import (
    add_atr_feature_pack_duckdb, add_mas_duckdb , add_emas_duckdb
)

In [111]:
from utils.bt_utils import ATRRiskSizer, ATRTrailingStopMixin, RegimeFilterMixin

In [112]:
from utils.performance import (
    run_backtest_daily, summarize_performance,
    transactions_to_df, extract_exec_df_from_strategy, 
    execs_to_lw_markers, plot_equity_curves_plotly,
    plot_returns_histogram, plot_drawdown_curve,
    compare_perfs, style_metrics, run_backtest_daily
)

In [113]:
from strategies import ATRBreakout, ATRPullback, KeltnerTrend

In [114]:
DB_MINUTE = PROJECT_ROOT / "data" / "processed" / "alpaca" / "price_minute_alpaca.duckdb"
print(f"DB_MINUTE: {DB_MINUTE}")
con_minute = duckdb.connect(str(DB_MINUTE), read_only=True)
tables = [t[0] for t in con_minute.execute("SHOW TABLES").fetchall()]
print("📋 Tables:", tables)


DB_MINUTE: /Users/luyanda/workspace/QuantTrade/data/processed/alpaca/price_minute_alpaca.duckdb
📋 Tables: ['alpaca_minute']


In [115]:
DB_DAILY = PROJECT_ROOT / "data" / "processed" / "dolt" / "stocks.duckdb"
print(f"DB_DAILY: {DB_DAILY}")
con_daily = duckdb.connect(str(DB_DAILY))
tables = [t[0] for t in con_daily.execute("SHOW TABLES").fetchall()]
print("📋 Tables:", tables)

DB_DAILY: /Users/luyanda/workspace/QuantTrade/data/processed/dolt/stocks.duckdb
📋 Tables: ['dividend', 'ohlcv', 'split', 'symbol']


In [116]:
ETFS = ["SPY", "QQQ"]
CHARTS_DIR = PROJECT_ROOT / "charts" / "004_ATR_Volatility"

In [117]:
START_CASH = 10_000
COMMISSION_BPS = 0.5 / 10_000  # 0.5 bps

## 2. Helpers

## 3. Data Ingestion

In [118]:
# --- Ingest latest minute-level data ---
minute_data = {sym: to_bt_minute_duckdb(con_minute, "alpaca_minute", sym) for sym in ETFS}

for symbol in ETFS:
    print(minute_data[symbol].tail(1))

                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:36:00  644.34  644.34  644.34  644.34   100.0          1.0   

                       vwap  
datetime                     
2025-08-13 14:36:00  644.34  
                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:55:00  582.45  582.45  582.45  582.45   340.0          2.0   

                       vwap  
datetime                     
2025-08-13 14:55:00  582.45  


In [119]:
# --- Ingest latest daily data ---
daily_data = {
    sym: to_bt_daily_duckdb(con_daily, sym, table="ohlcv", date_col="date", symbol_col="act_symbol")
    for sym in ETFS
}

for symbol in ETFS:
    print(daily_data[symbol].tail(1))


              open    high     low   close      volume
datetime                                              
2025-08-14  642.79  645.62  642.34  644.95  59327466.0
              open    high     low   close      volume
datetime                                              
2025-08-14  578.28  581.88  577.91  579.89  45425043.0


## 4. Data Quality Checks

U.S. Market (SPY, QQQ)
Assuming regular NYSE/Nasdaq trading hours:

| **Session**     | **Hours (ET)**   | **Duration** |
| --------------- | ---------------- | ------------ |
| Regular session | 09:30 – 16:00 ET | 6.5 hours    |
|                 |                  | 390 minutes  |

Expect around 390 rows per ETF

In [120]:
for symbol in ETFS:
    df = minute_data[symbol]
    print(f"\n🔍 {symbol}")
    print(f"  • Rows: {len(df)}")
    print(f"  • Date Range: {df.index.min().date()} → {df.index.max().date()}")
    print(f"  • Timezone-aware: {df.index.tz is not None}")
    # print(f"  • Missing 'close': {df['close'].isna().sum()}")

    # --- Drop timezone if needed ---
    df = df.copy()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    # --- Identify all available intraday dates ---
    df["date"] = df.index.normalize()
    available_dates = df["date"].unique()

    # --- Construct full expected range (business days) ---
    expected_dates = pd.date_range(
        start=df.index.min().normalize(),
        end=df.index.max().normalize(),
        freq='B'
    )

    # --- Missing trading days entirely ---
    missing_dates = sorted(set(expected_dates) - set(available_dates))
    print(f"  • Missing Intraday Dates: {len(missing_dates)}")
    # if missing_dates:
    #     print("    Example:", missing_dates[:5])

    # --- Check for partial trading days (fewer than 390 rows) ---
    counts = df.groupby("date").size()
    partial_days = counts[counts < 390]
    print(f"  • Partial Intraday Days (<390 rows): {len(partial_days)}")
    # if not partial_days.empty:
    #     print("    Example:", partial_days.head())



🔍 SPY
  • Rows: 192320
  • Date Range: 2023-08-09 → 2025-08-13
  • Timezone-aware: False
  • Missing Intraday Dates: 22
  • Partial Intraday Days (<390 rows): 279

🔍 QQQ
  • Rows: 187087
  • Date Range: 2023-08-09 → 2025-08-13
  • Timezone-aware: False
  • Missing Intraday Dates: 22
  • Partial Intraday Days (<390 rows): 306


## 5. Feature Engineering

In [ ]:
IND_MA_WINDOWS = [200]
IND_EMA_WINDOWS = [20]

In [122]:
print(daily_data["QQQ"].tail(1))

              open    high     low   close      volume
datetime                                              
2025-08-14  578.28  581.88  577.91  579.89  45425043.0


In [123]:
daily_data_ema = add_emas_duckdb(daily_data, con_daily, windows=IND_EMA_WINDOWS, price_col="close", prefix="ema")
daily_data_ema_ma = add_mas_duckdb(daily_data_ema, con_daily, windows=IND_MA_WINDOWS, price_col="close", prefix="ma")
daily_data_atr = add_atr_feature_pack_duckdb(daily_data_ema_ma, con_daily)

/Users/luyanda/workspace/QuantTrade/utils/features.py:410: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/Users/luyanda/workspace/QuantTrade/utils/features.py:415: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [124]:
print(daily_data_atr["QQQ"].tail(1))

              open    high     low   close      volume       ema20     ma200  \
datetime                                                                       
2025-08-14  578.28  581.88  577.91  579.89  45425043.0  567.696789  514.3993   

              tr     atr14    natr14  ...  kc20_basis      kc20_u      kc20_l  \
datetime                              ...                                       
2025-08-14  3.97  6.296785  0.010859  ...  567.696789  580.643775  554.749803   

             hh_20   ll_20  atr_stop_long_3  atr_stop_short_3     chop14  \
datetime                                                                   
2025-08-14  583.32  551.68       560.999645         54.871817  42.780941   

            bb_bw20  atr_vol_regime  
datetime                             
2025-08-14  0.04788             med  

[1 rows x 21 columns]


## 6. Strategy

In [125]:
# Bridge pandas DataFrame → Backtrader DataFeed

class VolatilityPandasData(bt.feeds.PandasData):
    lines = (
        'atr14','natr14',
        'kc20_basis','kc20_u','kc20_l',
        'hh_20','ll_20','hh_10','ll_10',
        'atr_stop_long_3','atr_stop_short_3',
        'chop14','bb_bw20',
    )
    params = dict(
        datetime=None, open='open', high='high', low='low', close='close',
        volume='volume', openinterest=-1,

        atr14='atr14', natr14='natr14',
        kc20_basis='kc20_basis', kc20_u='kc20_u', kc20_l='kc20_l',

        # present in your df
        hh_20='hh_20', ll_20='ll_20',

        # OPTIONAL: set to -1 so Backtrader doesn’t look for missing cols
        hh_10=-1, ll_10=-1,

        # often optional too
        atr_stop_long_3='atr_stop_long_3',
        atr_stop_short_3=-1,
        chop14=-1, bb_bw20=-1,
    )


In [126]:
SYMBOL = 'QQQ'
START_CASH = 100_000
COMMISSION_BPS = 0.5  # 0.5 bps = 0.005% (adjust to your IBKR model)

In [127]:
df = daily_data_atr["QQQ"].copy()

### Variant 1: — ATR Breakout (with ATR risk sizing)

In [ ]:
btres_atr_bo = run_backtest_daily(
    df=df,                                  # df already has: atr14, hh_20, ll_20, kc20_*, etc.
    strategy_cls=ATRBreakout,
    strategy_params=dict(
        hh_line='hh_20',
        ll_line='ll_20',                     # use ll_20 if you didn’t build ll_10
        natr_min=0.008,                      # gate signals to avoid micro-vol
        stop_line='atr_stop_long_3',         # from your feature pack
        printlog=False,
    ),
    start_cash=START_CASH,
    commission_bps=COMMISSION_BPS,
    datafeed_cls=VolatilityPandasData,
    symbol=SYMBOL,

    # if your runner supports custom sizers:
    sizer_cls=ATRRiskSizer,
    sizer_params=dict(risk_cash=200.0, m_atr=3.0, min_shares=1),
)

In [129]:
exec_df_atr_bo = transactions_to_df(btres_atr_bo.strategy.analyzers.tx)
markers_atr_bo = execs_to_lw_markers(exec_df_atr_bo)

In [130]:
# def build_markers(tx_df):
#     tx_df = tx_df.copy()
#     tx_df["dt"] = pd.to_datetime(tx_df["dt"]).dt.tz_localize(None)
#     buys  = tx_df[tx_df["side"] == "BUY"]["dt"].dt.strftime("%Y-%m-%d")
#     sells = tx_df[tx_df["side"] == "SELL"]["dt"].dt.strftime("%Y-%m-%d")
#     markers = (
#         [{"time": t, "position": "belowBar", "shape": "arrowUp", "text": "BUY"}  for t in buys] +
#         [{"time": t, "position": "aboveBar", "shape": "arrowDown","text": "SELL"} for t in sells]
#     )
#     return markers

# markers_atr = build_markers(exec_df_atr)

In [131]:
render_lightweight_chart(
    df=df,
    symbol=SYMBOL,
    title=f"{SYMBOL} — ATR Breakout",
    out_html=CHARTS_DIR/"etf_daily_atr_breakout_QQQ.html",
    theme="dark",
    height=720,
    ma_windows=IND_MA_WINDOWS,                 # optional baseline
    ema_windows=IND_EMA_WINDOWS,                 # if your helper supports EMA windows
    rsi_period=None,                  # keep RSI off if you don’t want the pane
    extra_lines={
        "kc_basis": "kc20_basis",
        "hh_20": "hh_20",
        "ll_20": "ll_20",
        "atr_stop": "atr_stop_long_3",
    },
    bands=[                           # upper/lower envelopes
        {"name": "Keltner 20,2xATR", "upper": "kc20_u", "lower": "kc20_l"},
    ],
    markers=markers_atr_bo,
    assets_rel="../../utils/static",
)

PosixPath('/Users/luyanda/workspace/QuantTrade/charts/004_ATR_Volatility/etf_daily_atr_breakout_QQQ.html')

### Variant 2: Keltner Trend-Follow

In [132]:
btres_kc = run_backtest_daily(
    df=df,
    strategy_cls=KeltnerTrend,
    strategy_params=dict(
        kc_u='kc20_u', kc_l='kc20_l', basis='kc20_basis',
        trend_ema_fast=20, trend_ema_slow=50,  # extra trend filter
        use_trend_filter=True,
        natr_min=0.0,                          # or 0.008
        stop_line='atr_stop_long_3',
        printlog=False,
    ),
    start_cash=START_CASH,
    commission_bps=COMMISSION_BPS,
    datafeed_cls=VolatilityPandasData,
    symbol=SYMBOL,
    
    sizer_cls=ATRRiskSizer,
    sizer_params=dict(risk_cash=200.0, m_atr=3.0),
)


In [133]:
exec_df_kc = transactions_to_df(btres_kc.strategy.analyzers.tx)
markers_kc = execs_to_lw_markers(exec_df_kc)

In [134]:
render_lightweight_chart(
    df=df,
    symbol=SYMBOL,
    title=f"{SYMBOL} — Keltner Trend-Follow",
    out_html=CHARTS_DIR/"etf_daily_atr_kc_QQQ.html",
    theme="dark",
    height=720,
    ma_windows=IND_MA_WINDOWS,                 # optional baseline
    ema_windows=IND_EMA_WINDOWS,                 # if your helper supports EMA windows
    rsi_period=None,                  # keep RSI off if you don’t want the pane
    extra_lines={
        "kc_basis": "kc20_basis",
        "hh_20": "hh_20",
        "ll_20": "ll_20",
        "atr_stop": "atr_stop_long_3",
    },
    bands=[                           # upper/lower envelopes
        {"name": "Keltner 20,2xATR", "upper": "kc20_u", "lower": "kc20_l"},
    ],
    markers=markers_kc,
    assets_rel="../../utils/static",
)

PosixPath('/Users/luyanda/workspace/QuantTrade/charts/004_ATR_Volatility/etf_daily_atr_kc_QQQ.html')

### Variant 3: ATR Pullback in Uptrend

In [ ]:
btres_atr_pb = run_backtest_daily(
    df=df,
    strategy_cls=ATRPullback,
    strategy_params=dict(
        base_ema=20, trend_sma=200,           # uptrend + confirmation EMA
        pullback_atr=1.5,                     # depth threshold in ATRs
        confirm_above_ema=True,
        natr_min=0.0,                         # or 0.008
        stop_line='atr_stop_long_3',
        printlog=False,
    ),
    start_cash=START_CASH,
    commission_bps=COMMISSION_BPS,
    datafeed_cls=VolatilityPandasData,
    symbol=SYMBOL,
    sizer_cls=ATRRiskSizer,
    sizer_params=dict(risk_cash=200.0, m_atr=3.0),
)


In [136]:
exec_df_pb = transactions_to_df(btres_atr_pb.strategy.analyzers.tx)
markers_pb = execs_to_lw_markers(exec_df_pb)

In [137]:
render_lightweight_chart(
    df,
    symbol="QQQ",
    out_html=CHARTS_DIR/"etf_daily_atr_pb_QQQ.html",
    theme="dark",
    ma_windows=IND_MA_WINDOWS,  
    # ema_windows=IND_EMA_WINDOWS,          
    # rsi_period=IND_RSI_PERIOD,             
    # rsi_bounds=IND_RSI_BOUNDS,
    timeframes=["1d", "1h", "15m"],
    default_tf="1d",
    watermark_text="QQQ — {tf}",
    watermark_opacity=0.0001,
    assets_rel="../../utils/static",
    markers=markers_pb,
)

PosixPath('/Users/luyanda/workspace/QuantTrade/charts/004_ATR_Volatility/etf_daily_atr_pb_QQQ.html')

## 7. Performance Evaluation

In [138]:
# Run your three backtests (already done), then summarize each:
perf_atr_bo = summarize_performance(btres_atr_bo)
perf_kc = summarize_performance(btres_kc)
perf_atr_pb = summarize_performance(btres_atr_pb)

# Vectorized comparison
fig_eq, fig_hist, fig_dd, metrics_df = compare_perfs(
    {
        "ATR Breakout": perf_atr_bo,
        "Keltner Trend-Follow": perf_kc,
        "ATR Pullback in Uptrend": perf_atr_pb
    },
    symbol="SPY",
    include_bh=True   # overlays Buy&Hold once
)

fig_eq.show()
fig_hist.show()
fig_dd.show()
style_metrics(metrics_df)


/Users/luyanda/workspace/QuantTrade/utils/performance.py:225: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/Users/luyanda/miniforge3/lib/python3.9/site-packages/plotly/io/_renderers.py:51: UserWarning:

Plotly version >= 6 requires Jupyter Notebook >= 7 but you have 6.4.12 installed.
 To upgrade Jupyter Notebook, please run `pip install notebook --upgrade`.



,Start,End,Start Value,End Value,Total Return %,CAGR %,Sharpe,Sortino,Sharpe (BT),MaxDD %,Win rate %,Trades,Wins,Losses,Gross PnL,Net PnL,Avg Trade (Net),Longest win streak,Longest loss streak
Strategy,,,,,,,,,,,,,,,,,,,
ATR Pullback in Uptrend,2011-01-03,2025-08-14,"100,000",nan,nan%,nan%,0.55,0.62,nan,-23791407.90%,33.333333,18.000000,6.000000,11.000000,1025.798142,857.053434,50.414908,nan,nan
Keltner Trend-Follow,2011-01-03,2025-08-14,"100,000","119,925",19.92%,1.25%,0.21,0.15,0.01,-14.73%,44.444444,81.000000,36.000000,44.000000,20506.114236,19598.886397,244.986080,nan,nan
ATR Breakout,2011-01-03,2025-08-14,"100,000","100,000",0.00%,0.00%,nan,nan,nan,0.00%,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan


In [139]:
con_minute.close()

In [140]:
con_daily.close()